# Predictive Modeling

<img src="../../reports/figures/logo_cmsr92.png" alt="CMSR92" height="60" style="display:block;margin:auto;">

**Modelado predictivo profesional para proyecciones y pricing.**

> Visualizaciones y explicaciones usan colores neutros y profesionales, con el logo CMSR92 solo en la cabecera.

In [1]:
import os, pandas as pd
from pathlib import Path
import sys

# Resolver raíz del proyecto de forma robusta

def find_root(start=None):
    p = Path(start or Path.cwd()).resolve()
    for _ in range(6):
        if (p / 'requirements.txt').exists() and (p / 'src').exists():
            return p
        if (p / '.git').exists() and (p / 'src').exists():
            return p
        p = p.parent
    return Path.cwd()

ROOT = find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.modeling import train_retention_model, train_frequency_model, save_model

FEAT_PATH = ROOT / 'data' / 'processed' / 'model_features.csv'

print('cwd:', os.getcwd())
print('ROOT:', ROOT)
print('Features existe?', FEAT_PATH.exists(), '->', FEAT_PATH)

try:
    feat = pd.read_csv(FEAT_PATH)
    # Asegurar que existe target Purchase_Again
    assert 'Purchase_Again' in feat.columns, 'Falta la columna Purchase_Again en features.'

    # Modelo de retención
    clf, rep = train_retention_model(feat, target='Purchase_Again')
    print('Retención ->', rep)
    save_model(clf, 'retention_model.pkl')

    # Modelo de frecuencia (simulación: usar Ticket_Price como proxy de visitas esperadas solo para demo)
    y_reg = (feat['Ticket_Price'] / feat['Ticket_Price'].median()).clip(lower=0.2, upper=4.0)
    reg, reg_metrics = train_frequency_model(feat.drop(columns=['Purchase_Again']), y_reg)
    print('Frecuencia ->', reg_metrics)
    save_model(reg, 'frequency_model.pkl')
except Exception as e:
    print('AVISO:', e)

cwd: C:\Users\Carlos\OneDrive\Documents\GitHub\cine\notebooks
ROOT: C:\Users\Carlos\OneDrive\Documents\GitHub\cine
Features existe? True -> C:\Users\Carlos\OneDrive\Documents\GitHub\cine\data\processed\model_features.csv


Retención -> ClassificationReport(accuracy=0.48322147651006714, precision=0.45454545454545453, recall=0.4225352112676056, f1=0.43795620437956206, roc_auc=0.41197183098591544)
Frecuencia -> {'r2': 0.9999932476840412, 'mae': 0.00048270981465418255}


In [2]:
# Variables más relevantes para la predicción de retención
import matplotlib.pyplot as plt
import numpy as np

# Diccionario de mapeo de nombres de variables a español profesional
nombre_vars = {
    'Ticket_ID': 'ID de ticket',
    'Age': 'Edad',
    'Ticket_Price': 'Precio del ticket',
    'Number_of_Person': 'Personas por compra',
    'gasto_total_est': 'Gasto total',
    'gasto_promedio': 'Gasto promedio',
    'Movie_Genre_Comedy': 'Comedia',
    'Movie_Genre_Drama': 'Drama',
    'Movie_Genre_Horror': 'Terror',
    'Movie_Genre_Sci-Fi': 'Ciencia ficción',
    'Seat_Type_Standard': 'Asiento estándar',
    'Seat_Type_Vip': 'Asiento VIP',
    'age_group_18-24': 'Edad 18-24',
    'age_group_25-39': 'Edad 25-39',
    'age_group_40-59': 'Edad 40-59',
    'age_group_60+': 'Edad 60+'
}

X = feat.drop(columns=['Purchase_Again'])
X_sample = X.sample(n=min(500, len(X)), random_state=42)

importances = clf.named_steps['clf'].feature_importances_
features = X.columns

# Traducir nombres
features_es = [nombre_vars.get(f, f) for f in features]
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 6))
plt.title('Variables más relevantes para la predicción de retención')
plt.barh(np.array(features_es)[indices], importances[indices], color='skyblue')
plt.xlabel('Importancia')
plt.gca().invert_yaxis()
plt.show()

for feat, imp in zip(np.array(features_es)[indices], importances[indices]):
    print(f'{feat}: {imp:.4f}')

Personas por compra: 0.2519
Gasto total: 0.2339
Edad: 0.2332
ID de ticket: 0.2232
Precio del ticket: 0.0579


C:\Users\Carlos\AppData\Local\Temp\ipykernel_15144\129685567.py:40: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
